In [1]:
from web3 import Web3
from solcx import install_solc, compile_standard
import solcx
import json
import os

install_solc(version='0.8.0')

onchain = Web3(Web3.HTTPProvider('http://localhost:8545')) #on-chain

s1 = Web3(Web3.HTTPProvider('http://localhost:8546')) #shard 1
s2 = Web3(Web3.HTTPProvider('http://localhost:8547')) #shard 2
s3 = Web3(Web3.HTTPProvider('http://localhost:8548')) #shard 3

onchain.isConnected(), s1.isConnected(), s2.isConnected(), s3.isConnected()

(True, True, False, False)

In [4]:
# codice per far inserire all'utente il path dello smart contract da deployare

print("Inserisci la path del file solidity dello smart contract di cui fare il deploy")
file_path = input()

# controllo che il file esista effettivamente e che sia di tipo .sol
while not os.path.exists(file_path) and os.path.splitext(file_path)[1] != ".sol":
    os.system('clear')
    print("La path deve esistere e il file deve essere di tipo .sol")
    file_path = input()
    
# Leggo il file contenente lo smart contract
with open(file_path, "r") as file:
    prova_on_chain_file = file.read()

Inserisci la path del file solidity dello smart contract di cui fare il deploy


In [10]:
try:
    # Compilo il file contenente lo smart contract
    compiled_sol = compile_standard(
        {
            "language": "Solidity",
            "sources": {"prova_on_chain.sol": {"content": prova_on_chain_file}},
            "settings": {
                "outputSelection": {
                    "*": {
                        "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"] # output needed to interact with and deploy contract 
                    }
                }
            },
        },
        solc_version="0.8.0",
    )

    # get bytecode
    bytecode = compiled_sol["contracts"]["prova_on_chain.sol"]["on_chain_manager"]["evm"]["bytecode"]["object"]
    # get abi
    abi = json.loads(compiled_sol["contracts"]["prova_on_chain.sol"]["on_chain_manager"]["metadata"])["output"]["abi"]

except:
    os.system('clear')
    print("La compilazione dello smart contract non è andata a buon fine")

SolcError: ParserError: Expected ';' but got '('
  --> prova_on_chain.sol:25:36:
   |
25 |     unction get_shard_where_deploy () public view returns (uint8 shard_number) {
   |                                    ^


> command: `/Users/andreacivitarese/.solcx/solc-v0.8.0 --standard-json`
> return code: `0`
> stdout:
{"errors":[{"component":"general","errorCode":"2314","formattedMessage":"ParserError: Expected ';' but got '('\n  --> prova_on_chain.sol:25:36:\n   |\n25 |     unction get_shard_where_deploy () public view returns (uint8 shard_number) {\n   |                                    ^\n\n","message":"Expected ';' but got '('","severity":"error","sourceLocation":{"end":1308,"file":"prova_on_chain.sol","start":1307},"type":"ParserError"}],"sources":{}}

> stderr: